In [ ]:
import os
import re
import numpy as np
import pdfplumber

from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

from openai import OpenAI
from dotenv import load_dotenv


In [ ]:
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("API Loaded Successfully")
